In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import os

In [3]:
# Create path to project folder
path = r'C:\Users\tatal\06-2023 Instacart Basket Analysis'

## Walk through excercise

In [5]:
#IMPORT ords_prods_mergedpickle from Prepared folder
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [10]:
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [15]:
df_ords_prods_merged.shape

(32404859, 14)

In [13]:
#create substet of 100000 rows to avoid memory issues
df = df_ords_prods_merged[:1000000]

In [12]:
df.shape

(1000000, 14)

In [16]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [17]:
# create if statment to evaluate the price of the item
def price_label(row): #define the funcition and give a name

  if row['prices'] <= 5: #first condition
    return 'Low-range product' #if meets the condition return value
  elif (row['prices'] > 5) and (row['prices'] <= 15): #repeat for second and thrid condition
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data' #if none condition is met

## Apply the if statment in the database

#f['price_range'] creates a new column to store the return values from if statment.apply the price_label function on axis=1. This axis = 1 stands for “rows,” so it’s essentially telling Python to apply the function to all rows within the dataframe. (Conversely, axis = 0 would refer to all columns within the dataframe.)

In [18]:
df['price_range'] = df.apply(price_label, axis=1)

C:\Users\tatal\AppData\Local\Temp\ipykernel_3312\2848743372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [19]:
# run the value_counts() function to check the values in your new column. 
df['price_range'].value_counts(dropna =  False)

Mid-range product    756450
Low-range product    243550
Name: price_range, dtype: int64

In [20]:
# there is no high value as we are workin with the first 1M rows. Check the high value of this subset
df['prices'].max()

14.8

## Use the loc function (preset function from pandas) to avoid the warning. Replaces the custom if

##define the locs on in each line

In [23]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [22]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [24]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [25]:
# run the value_counts() function to check the values in your new column. 
df['price_range_loc'].value_counts(dropna =  False)

Mid-range product    756450
Low-range product    243550
Name: price_range_loc, dtype: int64

#the loc function is way faster in that way it can be applied to the entire database

In [28]:
df_ords_prods_merged.loc[df_ords_prods_merged['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [30]:
df_ords_prods_merged.loc[(df_ords_prods_merged['prices'] <= 15) & (df_ords_prods_merged['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [32]:
df_ords_prods_merged.loc[df_ords_prods_merged['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [33]:
df_ords_prods_merged['price_range_loc'].value_counts(dropna =  False)

Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: price_range_loc, dtype: int64

## loops

In [34]:
##get the most buys day of the week by checking the frequency of days week column
df_ords_prods_merged['orders_day_of_week'].value_counts(dropna =  False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: orders_day_of_week, dtype: int64

## Create a column to indicate Busiest day,” “Least busy,” and “Regularly busy.”

In [45]:
# create a column to indicate the day
result = []

for value in df_ords_prods_merged["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [46]:
#print result
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Reg

In [47]:
# create the new column and assign it to result
df_ords_prods_merged["busiest_day"] = result

In [48]:
df_ords_prods_merged['busiest_day'].value_counts(dropna =  False)

Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: busiest_day, dtype: int64

## TASK 4.7

##Create  “Busiest days” (plural) This label should correspond with the two busiest days of the week as opposed to the single busiest day. At the same time, they’d also like to know the two slowest days

In [55]:
## Run a new loop to create a new result list with the 2 busiest and lowest day
new_result = []

for value in df_ords_prods_merged["orders_day_of_week"]:
  if value == 0 or value ==1:
    new_result.append("Busiest days")
  elif value == 3 or value == 4:
    new_result.append("Least busy days")
  else:
    new_result.append("Regularly busy days")

In [56]:
new_result

['Regularly busy days',
 'Least busy days',
 'Least busy days',
 'Least busy days',
 'Least busy days',
 'Regularly busy days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Least busy days',
 'Busiest days',
 'Regularly busy days',
 'Regularly busy days',
 'Busiest days',
 'Busiest days',
 'Regularly busy days',
 'Regularly busy days',
 'Least busy days',
 'Least busy days',
 'Least busy days',
 'Least busy days',
 'Least busy days',
 'Least busy days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy days',
 'Regularly busy days',
 'Busiest days',
 'Regularly busy days',
 'Regularly busy days',
 'Busiest days',
 'Regularly busy days',
 'Regularly busy days',
 'Regularly busy days',
 'Least busy days',
 'Regularly busy days',
 'Least busy days',
 'Busiest days',
 'Busiest days',
 'Regularly busy days',
 'Least busy days',
 'Least busy days',
 'Regularly busy days',
 'Regularly busy days',
 'Busiest days',
 'Busiest days',
 'Regularly busy days',
 'Busiest day

In [57]:
# create the new column and assign it to new_result
df_ords_prods_merged["Busiest_days"] = new_result

In [58]:
#check frequency of new column
df_ords_prods_merged['Busiest_days'].value_counts(dropna =  False)

Regularly busy days    12916111
Busiest days           11864412
Least busy days         7624336
Name: Busiest_days, dtype: int64

In [59]:
#check new headers
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days


In [60]:
#check shape
df_ords_prods_merged.shape

(32404859, 17)

## Task 4 - Identify the busiest hour of the day

In [61]:
## Check the frequency of order_hour_of_day of the day column
df_ords_prods_merged['order_hour_of_day'].value_counts(dropna =  False)

10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: order_hour_of_day, dtype: int64

In [79]:
#Create list with  of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create 3 periods of 8 hours.

period = []
for value in df_ords_prods_merged["order_hour_of_day"]:
  if value in [10,11,14,15,13,12,16,9]:
    period.append("most orders")
  elif value in [17,8,18,19,20,7,21,22]:
    period.append("average orders")
  else:
    period.append("fewest orders")

In [80]:
period

['average orders',
 'average orders',
 'most orders',
 'average orders',
 'most orders',
 'average orders',
 'most orders',
 'most orders',
 'most orders',
 'average orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'average orders',
 'most orders',
 'average orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'average orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'average orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'most orders',
 'average orders',
 'most orders',
 'most orders',
 'most ord

In [81]:
# create the new column and assign it to period
df_ords_prods_merged["busiest_period_of_day"] = period

In [82]:
#check frequencey of new column
df_ords_prods_merged['busiest_period_of_day'].value_counts(dropna =  False)

most orders       21118071
average orders     9997651
fewest orders      1289137
Name: busiest_period_of_day, dtype: int64

In [83]:
#check new headers
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,most orders


In [84]:
#check shape
df_ords_prods_merged.shape

(32404859, 18)

## EXPORT AS PICKLE

In [86]:
#export the merge data as pickle as it's too large
df_ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined_busy.pkl'))